<a href="https://colab.research.google.com/github/yyg00/itp303_full_stack_web_project/blob/master/interview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# self-attention
import torch
import torch.nn as nn
import math
# ref: https://github.com/zxuu/Self-Attention/blob/main/Decoder/My_Decoder.py， https://mdnice.com/writing/fc0b920d4ca84837a5712df1a46865d2
class SelfAttention(nn.Module):
  def __init__(self, d_model, d_qk, d_v, mask=None):
    # super(SelfAttention, self).__init__()
    super().__init__()
    self.d_model = d_model
    self.d_qk = d_qk
    self.d_v = d_v
    self.mask = mask
    self.initialize_Q = nn.Linear(d_model, d_qk, bias=False)
    self.initialize_K = nn.Linear(d_model, d_qk, bias=False)
    self.initialize_V = nn.Linear(d_model, d_v, bias=False)
    self._norm_fact = 1 / math.sqrt(self.d_qk) # or d_k(model) = k.size()[-1], or size(-1)
  def forward(self, x):
    # x: [batch, seq_len, d_model]
    # linear transformation of x for Q,K,V in self attention
    Q = self.initialize_Q(x) # [batch, seq_len, d_qk]
    K = self.initialize_K(x) # [batch, seq_len, d_qk]
    V = self.initialize_V(x) # [batch, seq_len, d_v]
    # print(Q.size())
    # print(V.size())
    attn_logits = torch.matmul(Q, K.transpose(-2,-1)) * self._norm_fact # [batch, seq_len, d_qk] => [batch, seq_len, seq_len]
    if self.mask is not None:
      print(attn_logits.size())
      attn_logits = attn_logits.masked_fill(self.mask == 0, -9e15)
    attn_scores = torch.softmax(attn_logits, dim=-1) # [batch, seq_len, seq_len]
    attn_values = torch.matmul(attn_scores, V) # [batch, seq_len, d_v]
    return attn_values, attn_scores
batch_size = 3
d_model = 6
seq_len = 4
d_v = 8
d_qk = 7
# x = [[[ 0.1532, -0.0044,  0.6197,  0.6332, -1.1813,  0.3393],
#          [ 0.4235, -0.1569,  0.1241,  0.0882,  0.2117,  1.0789],
#          [-0.1987, -0.2671, -0.2119, -0.3006, -0.1815,  0.0309],
#          [ 0.5800, -0.0540, -0.3753, -0.3589,  0.0209,  0.0198]],

#         [[ 0.6163, -0.1154, -0.4507,  0.0220, -0.2439, -0.2730],
#          [-0.2422, -0.2329,  0.4126, -0.1035, -0.7506,  0.6158],
#          [-1.1735,  0.1261,  0.1183, -0.5666, -0.0784,  1.1600],
#          [ 1.4221,  0.5371,  1.0822,  0.3618, -1.1405, -0.2908]],

#         [[-1.4205, -0.3030,  0.6546,  0.1172,  0.0054,  0.2997],
#          [ 0.1374,  0.1845,  0.0117, -0.2321, -0.5516, -0.0238],
#          [-0.2277,  0.2537, -0.0104, -0.1850,  0.3958,  0.7365],
#          [ 0.4063, -0.1534,  0.2482,  0.2756,  0.7651,  1.8565]]]
batch_size = 1
d_model = 5
seq_len = 3 #一句话三个词 每个token dim是2, 一个batch 1个句子
d_qk = 3
d_v = 3
mask = [[[1,1,1],
        [1,0, 0],
         [0,0,1]]]
# x = torch.tensor(x)
mask = torch.tensor(mask)
x_gen = torch.randn(batch_size, seq_len, d_model)
attention = SelfAttention(d_model, d_qk, d_v,mask=None)
res, w = attention(x_gen)
print(w)
print(res)
print(res.size())

tensor([[[0.3567, 0.3288, 0.3145],
         [0.3314, 0.3258, 0.3429],
         [0.3077, 0.3274, 0.3649]]], grad_fn=<SoftmaxBackward0>)
tensor([[[ 0.0036,  0.4110, -0.3547],
         [ 0.0031,  0.3927, -0.3383],
         [ 0.0032,  0.3774, -0.3216]]], grad_fn=<UnsafeViewBackward0>)
torch.Size([1, 3, 3])


In [ ]:
# class MultiheadAttention(SelfAttention):
import torch.nn.functional as F
class MultiheadAttention(nn.Module):
    # input : batch_size * seq_len * input_dim
    # q : batch_size * input_dim * dim_k
    # k : batch_size * input_dim * dim_k
    # v : batch_size * input_dim * dim_v
  def __init__(self, n_heads, d_model, q, k, v, mask=None, dropout=None):
    # super().__init__(d_model, d_qk, d_v)
    # self.n_heads = n_heads
    # d_k = d_v = d_model / n_heads
    super().__init__()
    self.n_heads = n_heads
    self.d_model = d_model
    self.d_qk = d_model // n_heads # qkv 同维度
    self.d_v = d_model // n_heads
    self.mask = mask
    self.dropout = dropout
    # self.q = q
    # self.k = k
    # self.v = v
    self.Q = nn.Linear(d_model, d_model, bias=False)(q)
    self.K = nn.Linear(d_model, d_model, bias=False)(k)
    self.V = nn.Linear(d_model, d_model, bias=False)(v)

  def self_attention(self, q, k, v, dropout=None, mask=None):
    d_k = k.size()[-1] # d_k=单词的embedding长度
    # print(d_k == self.d_qk)
    attn_logits = torch.matmul(q, k.transpose(-2,-1)) / math.sqrt(d_k) #bmm
    if mask is not None: # softmax 前 mask
      attn_logits = attn_logits.masked_fill(mask == 0, -1e9)
      print('masked:', attn_logits)
    attn = F.softmax(attn_logits, dim=-1) # torch.softmax also works, this one is module. F one is a function
    if dropout is not None:
      attn = dropout(attn)
    scores = torch.matmul(attn, v)
    return scores, attn

  def forward(self):
    '''
    q,k,v linear to Q, K, V: [batch_size, seq_len, d_model]
    '''
    batch_size = self.q.size()[0] # or size(0)
    # [b, seq_len, d_model] => [b, seq_len, heads, d_model // heads(head size)] => [b, heads, seq_len, split_emb_size]
    mq = self.Q.view(batch_size, -1, self.n_heads, self.d_qk).transpose(1, 2) # split embed vertically for multihead purpose, in this case view == reshape since store contiguously
    mk = self.K.view(batch_size, -1, self.n_heads, self.d_qk).transpose(1, 2)
    mv = self.V.view(batch_size, -1, self.n_heads, self.d_qk).transpose(1, 2)
    if self.mask is not None:
    # 多头注意力机制的线性变换层是4维，是把query[batch, frame_num or seq_len, d_model]变成[batch, -1, head, d_k(d_qk)]
    # 再1，2维交换变成[batch, head, -1, d_k], 所以mask要在第一维添加一维，与后面的self attention计算维度一样
      self.mask = self.mask.unsqueeze(1)
    '''
    # print(mq)
    # print(self.Q.size(), mq.size())
    # # mq = self.Q.reshape(batch_size, self.n_heads, self.d_qk).transpose(1, 2)
    '''
    x, attn = self.self_attention(mq, mk, mv) # scores: [b, heads, seq_len, split_emb_size] weights: [b, heads, seq_len, seq_len]
    # 变为三维， 或者说是concat head(拼接各个head-attention的结果)
    x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_qk) # [b, seq_len, d_model]
    # fc layer
    x = nn.Linear(d_model, d_model)(x) # [b, seq_len, d_model]
    return x
n_heads = 3
seq_len = 5
d_model = 12
batch_size = 1
q = torch.randn(batch_size, seq_len, d_model)
k = torch.randn(batch_size, seq_len, d_model)
v = torch.randn(batch_size, seq_len, d_model)
multi = MultiheadAttention(n_heads, d_model, q,k,v)
res = multi()
print(res)

tensor([[[ 0.2324,  0.2188,  0.1741, -0.3829, -0.3040,  0.1120, -0.0527,
          -0.1659,  0.0654, -0.2432,  0.1959,  0.0615],
         [ 0.2558,  0.2721,  0.0776, -0.4345, -0.2776,  0.1389, -0.0541,
          -0.1901,  0.0287, -0.2396,  0.1855,  0.0797],
         [ 0.2673,  0.2859,  0.0894, -0.4039, -0.2944,  0.1376, -0.0608,
          -0.1687,  0.0756, -0.2283,  0.1990,  0.0636],
         [ 0.2553,  0.2405,  0.1331, -0.3805, -0.2979,  0.1422, -0.0395,
          -0.1742,  0.0645, -0.2247,  0.2040,  0.0701],
         [ 0.2560,  0.2950,  0.1015, -0.3900, -0.2904,  0.1141, -0.0682,
          -0.1585,  0.0956, -0.2379,  0.1998,  0.0600]]],
       grad_fn=<ViewBackward0>)


In [ ]:
# batch norm 2D and 1D
import numpy as np
class BatchNorm2D():
    gamma, beta = 1, 0  # 缩放因子γ和平移因子β，能训练的参数
    def __init__(self, channel, momentum=0.1, eps=1e-5):
        self.running_mean = np.zeros(channel) # 用于测试时
        self.running_var = np.ones(channel)   # 同上
        self.momentum = momentum
        self.eps = eps                        # 接近于0的数，用于避免分母为0
        self.training = True

    def forward(self, input):
        # input.shape: (B, C, H, W)
        len_ch = input.size(1)
        output = np.zeros(input.size())

        for i in range(len_ch):
            in_ch = input[:, i, :, :]
            total_elem = in_ch.numel()

            if self.training:
                # 计算均值和方差，并归一化
                mean = in_ch.sum() / total_elem
                var = ((in_ch - mean) ** 2).sum() / total_elem
                out_ch = (in_ch - mean) / (var + self.eps) ** 0.5  # 归一化

                # 更新参数
                var_unbiased = ((in_ch - mean) ** 2).sum() / (total_elem - 1)
                self.running_mean[i] = self.running_mean[i] * (1 - self.momentum) + mean * self.momentum
                self.running_var[i] = self.running_var[i] * (1 - self.momentum) + var_unbiased * self.momentum
            else:
                out_ch = (in_ch - self.running_mean[i]) / (self.running_var[i] + self.eps) ** 0.5


            out_ch = self.gamma * out_ch + self.beta  # 缩放平移
            output[:, i, :, :] = out_ch

        return output
class BatchNorm1D():
    gamma, beta = 1, 0  # 缩放因子γ和平移因子β，能训练的参数
    def __init__(self, momentum=0.1, eps=1e-5):
        self.running_mean = 0
        self.running_var = 1
        self.momentum = momentum
        self.eps = eps
        self.training = True

    def forward(self, input):

        total_elem = input.numel()

        if self.training:
            # 计算均值和方差，并归一化
            mean = input.sum() / total_elem
            var = ((input - mean) ** 2).sum() / total_elem
            output = (input - mean) / (var + self.eps) ** 0.5

            # 更新参数
            var_unbiased = ((input - mean) ** 2).sum() / (total_elem - 1)
            self.running_mean = self.running_mean * (1 - self.momentum) + mean * self.momentum
            self.running_var = self.running_var * (1 - self.momentum) + var_unbiased * self.momentum
        else:
            output = (input - self.running_mean) / (self.running_var + self.eps) ** 0.5

        output = self.gamma * output + self.beta  # 缩放平移
        return output

In [ ]:
import torch

a = torch.tensor([
    [
        [1, 2, 3, 4],
        [5, 6, 7, 8]
    ],
    [
        [9, 10, 11, 12],
        [13, 14, 15, 16]
    ]
])

# Shape: (2, 2, 4)

res1 = a.view(2, -1, 2, 2).transpose(1, 2)
res2 = a.view(2, 2, -1, 2)

print(res1)
print(res2)
batch_size, seq_len, d_model = Q.size()
n_heads = 8  # 假设有8个头
d_k = d_model // n_heads
d_v = d_model // n_heads

# 1. 投影到多个头
WQ = torch.nn.Linear(d_model, d_model)  # 投影权重矩阵
WK = torch.nn.Linear(d_model, d_model)
WV = torch.nn.Linear(d_model, d_model)
Qs = WQ(Q).view(batch_size, seq_len, n_heads, d_k).transpose(1, 2)  # [batch_size, n_heads, seq_len, d_k]
Ks = WK(K).view(batch_size, seq_len, n_heads, d_k).transpose(1, 2)
Vs = WV(V).view(batch_size, seq_len, n_heads, d_v).transpose(1, 2)

# 2. 对每个头进行自注意力
# import torch.nn.functional as F

# d_k = Q.size(-1)  # 获取 d_k 的值
# weights = torch.einsum('bij,bjk->bik', Q, K.transpose(-2, -1)) / (d_k**0.5)
# normalized_weights = F.softmax(weights, dim=-1)
# output = torch.einsum('bij,bjk->bik', normalized_weights, V)

weights = torch.einsum('bhid,bhjd->bhij', Qs, Ks.transpose(-2, -1)) / (d_k ** 0.5)
normalized_weights = F.softmax(weights, dim=-1)
attention_output = torch.einsum('bhij,bhjd->bhid', normalized_weights, Vs)

# 3. 拼接所有头的输出
concat_attention = attention_output.transpose(1, 2).contiguous().view(batch_size, seq_len, d_model)

# 4. 通过一个线性层得到最终输出
W_out = torch.nn.Linear(d_model, d_model)
output = W_out(concat_attention)


tensor([[[[ 1,  2],
          [ 5,  6]],

         [[ 3,  4],
          [ 7,  8]]],


        [[[ 9, 10],
          [13, 14]],

         [[11, 12],
          [15, 16]]]])
tensor([[[[ 1,  2],
          [ 3,  4]],

         [[ 5,  6],
          [ 7,  8]]],


        [[[ 9, 10],
          [11, 12]],

         [[13, 14],
          [15, 16]]]])


In [ ]:
# from math import sqrt
# import torch
# import torch.nn


# class Self_Attention(nn.Module):
#     # input : batch_size * seq_len * input_dim
#     # q : batch_size * input_dim * dim_k
#     # k : batch_size * input_dim * dim_k
#     # v : batch_size * input_dim * dim_v
#     def __init__(self,input_dim,dim_k,dim_v):
#         super(Self_Attention,self).__init__()
#         self.q = nn.Linear(input_dim,dim_k)
#         self.k = nn.Linear(input_dim,dim_k)
#         self.v = nn.Linear(input_dim,dim_v)
#         self._norm_fact = 1 / sqrt(dim_k)


#     def forward(self,x):
#         Q = self.q(x) # Q: batch_size * seq_len * dim_k
#         K = self.k(x) # K: batch_size * seq_len * dim_k
#         V = self.v(x) # V: batch_size * seq_len * dim_v

#         atten = nn.Softmax(dim=-1)(torch.bmm(Q,K.permute(0,2,1))) * self._norm_fact # Q * K.T() # batch_size * seq_len * seq_len

#         output = torch.bmm(atten,V) # Q * K.T() * V # batch_size * seq_len * dim_v

#         return output
# att = Self_Attention(d_model, d_qk, d_v)
# rs2 = att(x_gen)
# print(rs2)

tensor([[[ 0.0092, -0.1427,  0.0127],
         [-0.2397, -0.2193,  0.0599],
         [-0.3094, -0.2240,  0.0623]]], grad_fn=<BmmBackward0>)


In [ ]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

             # Encoder_input    Decoder_input        Decoder_output
sentences = [['我 是 学 生 P' , 'S I am a student'   , 'I am a student E'],         # S: 开始符号
             ['我 喜 欢 学 习', 'S I like learning P', 'I like learning P E'],      # E: 结束符号
             ['我 是 男 生 P' , 'S I am a boy'       , 'I am a boy E']]             # P: 占位符号，如果当前句子不足固定长度用P占位 pad补0


src_vocab = {'P':0, '我':1, '是':2, '学':3, '生':4, '喜':5, '欢':6,'习':7,'男':8}   # 词源字典  字：索引
src_idx2word = {src_vocab[key]: key for key in src_vocab}
src_vocab_size = len(src_vocab)                 # 字典字的个数

tgt_vocab = {'S':0, 'E':1, 'P':2, 'I':3, 'am':4, 'a':5, 'student':6, 'like':7, 'learning':8, 'boy':9}
idx2word = {tgt_vocab[key]: key for key in tgt_vocab}                               # 把目标字典转换成 索引：字的形式
tgt_vocab_size = len(tgt_vocab)                                                     # 目标字典尺寸

src_len = len(sentences[0][0].split(" "))                                           # Encoder输入的最大长度 5
tgt_len = len(sentences[0][1].split(" "))                                           # Decoder输入输出最大长度 5

# 把sentences 转换成字典索引
def make_data(sentences):
    enc_inputs, dec_inputs, dec_outputs = [], [], []
    for i in range(len(sentences)):
      enc_input = [[src_vocab[n] for n in sentences[i][0].split()]]
      dec_input = [[tgt_vocab[n] for n in sentences[i][1].split()]]
      dec_output = [[tgt_vocab[n] for n in sentences[i][2].split()]]
      enc_inputs.extend(enc_input)
      dec_inputs.extend(dec_input)
      dec_outputs.extend(dec_output)
    return torch.LongTensor(enc_inputs), torch.LongTensor(dec_inputs), torch.LongTensor(dec_outputs)
enc_inputs, dec_inputs, dec_outputs = make_data(sentences)
print(enc_inputs)
print(dec_inputs)
print(dec_outputs)

'''
sentences 里一共有三个训练数据，中文->英文。把Encoder_input、Decoder_input、Decoder_output转换成字典索引，
例如"学"->3、“student”->6。再把数据转换成batch大小为2的分组数据，3句话一共可以分成两组，一组2句话、一组1句话。src_len表示中文句子
固定最大长度，tgt_len 表示英文句子固定最大长度。
'''
#自定义数据集函数
class MyDataSet(Data.Dataset):
  def __init__(self, enc_inputs, dec_inputs, dec_outputs):
    super(MyDataSet, self).__init__()
    self.enc_inputs = enc_inputs
    self.dec_inputs = dec_inputs
    self.dec_outputs = dec_outputs

  def __len__(self):
    return self.enc_inputs.shape[0]

  def __getitem__(self, idx):
    return self.enc_inputs[idx], self.dec_inputs[idx], self.dec_outputs[idx]

loader = Data.DataLoader(MyDataSet(enc_inputs, dec_inputs, dec_outputs), 2, True)

d_model = 512   # 字 Embedding 的维度
d_ff = 2048     # 前向传播隐藏层维度
d_k = d_v = 64  # K(=Q), V的维度
n_layers = 6    # 有多少个encoder和decoder
n_heads = 8     # Multi-Head Attention设置为8

# 位置嵌入，position Embedding
class PositionalEncoding(nn.Module):
    def __init__(self,d_model,dropout=0.1,max_len=5000):
        super(PositionalEncoding,self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pos_table = np.array([
        [pos / np.power(10000, 2 * i / d_model) for i in range(d_model)]
        if pos != 0 else np.zeros(d_model) for pos in range(max_len)])
        pos_table[1:, 0::2] = np.sin(pos_table[1:, 0::2])                  # 字嵌入维度为偶数时
        pos_table[1:, 1::2] = np.cos(pos_table[1:, 1::2])                  # 字嵌入维度为奇数时
        self.pos_table = torch.FloatTensor(pos_table)               # enc_inputs: [seq_len, d_model]
    def forward(self,enc_inputs):                                         # enc_inputs: [batch_size, seq_len, d_model]
        enc_inputs += self.pos_table[:enc_inputs.size(1),:]
        return self.dropout(enc_inputs)

'''
Mask句子中没有实际意义的占位符，例如’我 是 学 生 P’ ，P对应句子没有实际意义，所以需要被Mask，Encoder_input 和Decoder_input占位符
都需要被Mask。
这就是为了处理，句子不一样长，但是输入有需要定长，不够长的pad填充，但是计算又不需要这个pad，所以mask掉

这个函数最核心的一句代码是 seq_k.data.eq(0)，这句的作用是返回一个大小和 seq_k 一样的 tensor，只不过里面的值只有 True 和 False。如
果 seq_k 某个位置的值等于 0，那么对应位置就是 True，否则即为 False。举个例子，输入为 seq_data = [1, 2, 3, 4, 0]，
seq_data.data.eq(0) 就会返回 [False, False, False, False, True]
'''
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size()# seq_q 用于升维，为了做attention，mask score矩阵用的
    batch_size, len_k = seq_k.size()
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1) # 判断 输入那些含有P(=0),用1标记 ,[batch_size, 1, len_k]
    return pad_attn_mask.expand(batch_size,len_q,len_k) # 扩展成多维度   [batch_size, len_q, len_k]

'''
# Decoder输入Mask
用来Mask未来输入信息，返回的是一个上三角矩阵。比如我们在中英文翻译时候，会先把"我是学生"整个句子输入到Encoder中，得到最后一层的输出
后，才会在Decoder输入"S I am a student"（s表示开始）,但是"S I am a student"这个句子我们不会一起输入，而是在T0时刻先输入"S"预测，
预测第一个词"I"；在下一个T1时刻，同时输入"S"和"I"到Decoder预测下一个单词"am"；然后在T2时刻把"S,I,am"同时输入到Decoder预测下一个单
词"a"，依次把整个句子输入到Decoder,预测出"I am a student E"。
'''
def get_attn_subsequence_mask(seq):                               # seq: [batch_size, tgt_len]
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]          # 生成上三角矩阵,[batch_size, tgt_len, tgt_len]
    subsequence_mask = np.triu(np.ones(attn_shape), k=1)
    subsequence_mask = torch.from_numpy(subsequence_mask).byte()  #  [batch_size, tgt_len, tgt_len]
    return

# 计算注意力信息、残差和归一化
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):                             # Q: [batch_size, n_heads, len_q, d_k]
                                                                       # K: [batch_size, n_heads, len_k, d_k]
                                                                       # V: [batch_size, n_heads, len_v(=len_k), d_v]
                                                                       # attn_mask: [batch_size, n_heads, seq_len, seq_len]
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)   # scores : [batch_size, n_heads, len_q, len_k]
        scores.masked_fill_(attn_mask, -1e9)                           # 如果是停用词P就等于 0 在原tensor修改
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)                                # [batch_size, n_heads, len_q, d_v]
        return context, attn

# 多头自注意力机制
# 拼接之后 输入fc层 加入残差 Norm
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)

    def forward(self, input_Q, input_K, input_V, attn_mask):    # input_Q: [batch_size, len_q, d_model]
                                                                # input_K: [batch_size, len_k, d_model]
                                                                # input_V: [batch_size, len_v(=len_k), d_model]
                                                                # attn_mask: [batch_size, seq_len, seq_len]
        residual, batch_size = input_Q, input_Q.size(0)
        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # Q: [batch_size, n_heads, len_q, d_k]
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)              # attn_mask : [batch_size, n_heads, seq_len, seq_len]
        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)          # context: [batch_size, n_heads, len_q, d_v]
                                                                                 # attn: [batch_size, n_heads, len_q, len_k]
        # 拼接多头的结果
        context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v) # context: [batch_size, len_q, n_heads * d_v]
        output = self.fc(context)                                                # [batch_size, len_q, d_model]
        return nn.LayerNorm(d_model)(output + residual), attn


'''
## 前馈神经网络
输入inputs ，经过两个全连接层，得到的结果再加上 inputs （残差），再做LayerNorm归一化。LayerNorm归一化可以理解层是把Batch中每一句话
进行归一化。
'''
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False))

    def forward(self, inputs):                             # inputs: [batch_size, seq_len, d_model]
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model)(output + residual)   # [batch_size, seq_len, d_model]

## encoder layer(block)
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()                                     # 多头注意力机制
        self.pos_ffn = PoswiseFeedForwardNet()                                        # 前馈神经网络

    def forward(self, enc_inputs, enc_self_attn_mask):                                # enc_inputs: [batch_size, src_len, d_model]
        #输入3个enc_inputs分别与W_q、W_k、W_v相乘得到Q、K、V                             # enc_self_attn_mask: [batch_size, src_len, src_len]
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs,    # enc_outputs: [batch_size, src_len, d_model],
                                               enc_self_attn_mask)                    # attn: [batch_size, n_heads, src_len, src_len]
        enc_outputs = self.pos_ffn(enc_outputs)                                       # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs, attn

'''
## Encoder
第一步，中文字索引进行Embedding，转换成512维度的字向量。
第二步，在子向量上面加上位置信息。
第三步，Mask掉句子中的占位符号。
第四步，通过6层的encoder（上一层的输出作为下一层的输入）。
'''
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(src_vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList(
            [EncoderLayer() for _ in range(n_layers)]
        )

    def forward(self, enc_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        '''
        enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs) # [batch_size, src_len, src_len]
        enc_self_attns = []
        for layer in self.layers:
            # enc_outputs: [batch_size, src_len, d_model], enc_self_attn: [batch_size, n_heads, src_len, src_len]
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns




# 测试
'''
enc_inputs: 每个单词在字典当中的位置。下面是三句话，每个数字代表：该位置上的单词在字典中的位置。
tensor([[1, 2, 3, 4, 0],
        [1, 5, 6, 3, 7],
        [1, 2, 8, 4, 0]])
'''
enc_outputs, enc_self_attns = Encoder()(enc_inputs)
print(enc_outputs.shape)    # torch.Size([3, 5, 512])
